### TRAIN_TEST_SPLIT

In [1]:
import os
import random
import shutil

# Define paths
birds_folder = "BIRDS1"
split_folder = "BIRDS1_split"
train_folder = os.path.join(split_folder, "train")
test_folder = os.path.join(split_folder, "test")

# Create split folder if it doesn't exist
os.makedirs(split_folder, exist_ok=True)

# Create train and test folders inside split folder
os.makedirs(train_folder, exist_ok=True)
os.makedirs(test_folder, exist_ok=True)

# Get list of bird class folders
bird_class_folders = [folder for folder in os.listdir(birds_folder) if os.path.isdir(os.path.join(birds_folder, folder))]

# Create class folders in train and test folders
for bird_class_folder in bird_class_folders:
    train_class_folder = os.path.join(train_folder, bird_class_folder)
    test_class_folder = os.path.join(test_folder, bird_class_folder)
    os.makedirs(train_class_folder, exist_ok=True)
    os.makedirs(test_class_folder, exist_ok=True)


# Split dataset into train and test
for bird_class_folder in bird_class_folders:
    class_images = os.listdir(os.path.join(birds_folder, bird_class_folder))
    random.shuffle(class_images)  # Shuffle images
    split_index = int(0.7 * len(class_images))  # 80% train, 20% test

    # Randomly split images into train and test
    train_images = class_images[:split_index]
    test_images = class_images[split_index:]

    # Move images to train folder
    for image in train_images:
        src = os.path.join(birds_folder, bird_class_folder, image)
        dst = os.path.join(train_folder, bird_class_folder, image)
        if not os.path.exists(dst):
            shutil.copy(src, dst)

    # Move images to test folder
    for image in test_images:
        src = os.path.join(birds_folder, bird_class_folder, image)
        dst = os.path.join(test_folder, bird_class_folder, image)
        if not os.path.exists(dst):
            shutil.copy(src, dst)

print("Dataset split into train and test folders successfully.")

Dataset split into train and test folders successfully.


In [ ]:
import replicate
import os
os.environ["REPLICATE_API_TOKEN"] = "r8_eXJyxamZbCUsiyR5qRsTcZ9CM34Cywc1w7e87"
output = replicate.run(
    "microsoft/bringing-old-photos-back-to-life:c75db81db6cbd809d93cc3b7e7a088a351a3349c9fa02b6d393e35e0d51ba799",
    input={
        "HR": False,
        "image": "https://handegallery.com/displayimage.php?pid=10034&fullsize=1",
        "with_scratch": False
    }
)
print(output)

In [4]:
import tensorflow as tf
def reset_gpu_memory():
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            # Currently, memory growth needs to be the same across GPUs
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
        except RuntimeError as e:
            # Memory growth must be set before GPUs have been initialized
            print(e)

# Call the function to reset GPU memory
reset_gpu_memory()

2024-02-03 18:39:39.880700: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-03 18:39:40.118746: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-03 18:39:40.118825: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-03 18:39:40.157591: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-03 18:39:40.236303: I tensorflow/core/platform/cpu_feature_guar

1 Physical GPUs, 1 Logical GPUs


2024-02-03 18:39:42.063261: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-03 18:39:42.092800: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-03 18:39:42.093001: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [8]:
from keras.src.preprocessing.image import ImageDataGenerator

train_data_dir = "BIRDS1_split/train"
validation_data_dir = "BIRDS1_split/test"
img_height, img_width = 299, 299
batch_size = 32
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.5,
    zoom_range=0.5,
    width_shift_range=0.5,
    height_shift_range=0.5,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')


Found 9404 images belonging to 68 classes.
Found 4069 images belonging to 68 classes.


In [9]:

import os
import tensorflow as tf
from tensorflow.keras import regularizers
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, AveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.regularizers import l2
from tensorflow import keras
from tensorflow.keras import models
import cv2

In [ ]:

from tensorflow.keras.layers import BatchNormalization

inception = InceptionV3(weights='imagenet', include_top=False)
x = inception.output
x = GlobalAveragePooling2D()(x)
x = Dense(256,activation='relu')(x)
x = Dense(256,activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

predictions = Dense(68,kernel_regularizer=regularizers.l2(0.005), activation='softmax')(x)

model = Model(inputs=inception.input, outputs=predictions)
model.compile(optimizer=SGD(lr=0.0001, momentum=0.75), loss='categorical_crossentropy', metrics=['accuracy'])
checkpointer = ModelCheckpoint(filepath='best_model_3class.hdf5', verbose=1, save_best_only=True)
csv_logger = CSVLogger('history_3class.log')

history = model.fit_generator(train_generator,
                    steps_per_epoch = 9404 // batch_size,
                    validation_data=validation_generator,
                    validation_steps=4069 // batch_size,
                    epochs=30,
                    verbose=1,
                    callbacks=[csv_logger, checkpointer])

model.save('model_trained_3class.hdf5')

/tmp/ipykernel_12497/2326467380.py:18: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator,


Epoch 1/30


2024-02-03 19:15:57.318823: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.77GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-02-03 19:15:57.442829: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.77GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


 16/293 [>.............................] - ETA: 1:19 - loss: 5.3499 - accuracy: 0.0312

2024-02-03 19:16:16.856588: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.75GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-02-03 19:16:16.970398: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.75GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


 71/293 [======>.......................] - ETA: 1:47 - loss: 4.8700 - accuracy: 0.0666

In [1]:
from tensorflow.keras.models import Sequential, Model
import tensorflow as tf
model_97 = tf.keras.models.load_model('Best_models/val_accuracy_96.95_inceptionv3.hdf5')
model_97.summary()

2024-02-11 19:29:56.333739: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-11 19:29:56.554457: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-11 19:29:56.554538: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-11 19:29:56.591700: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-11 19:29:56.671949: I tensorflow/core/platform/cpu_feature_guar

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, None, None, 3)]      0         []                            
                                                                                                  
 conv2d_94 (Conv2D)          (None, None, None, 32)       864       ['input_2[0][0]']             
                                                                                                  
 batch_normalization_94 (Ba  (None, None, None, 32)       96        ['conv2d_94[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 activation_94 (Activation)  (None, None, None, 32)       0         ['batch_normalization_94

In [5]:
import utils

image = utils.load_prepare_image_in_tf("images/Lewiss_Woodpecker-1.jpg", img_shape=224)
image

<tf.Tensor: shape=(224, 224, 3), dtype=float32, numpy=
array([[[0.50152063, 0.4387755 , 0.37995198],
        [0.53321326, 0.47046816, 0.41164464],
        [0.5338735 , 0.47112846, 0.40838337],
        ...,
        [0.4296924 , 0.37479043, 0.33165315],
        [0.4318227 , 0.37692073, 0.33378348],
        [0.44921345, 0.3943115 , 0.35117424]],

       [[0.50764304, 0.44489795, 0.38607442],
        [0.5216286 , 0.45888355, 0.40006003],
        [0.52535516, 0.46261004, 0.39986494],
        ...,
        [0.4502947 , 0.39539275, 0.3522555 ],
        [0.42917642, 0.37427446, 0.3311372 ],
        [0.44351253, 0.38861057, 0.34547332]],

       [[0.53635955, 0.47361445, 0.4147909 ],
        [0.5335434 , 0.47079828, 0.41197476],
        [0.52245903, 0.45971388, 0.39696875],
        ...,
        [0.43299368, 0.37809172, 0.33495447],
        [0.435849  , 0.38094702, 0.33780977],
        [0.43562436, 0.3807224 , 0.33758515]],

       ...,

       [[0.7863973 , 0.7863973 , 0.7863973 ],
        [0.89

In [30]:
import os
labels = os.listdir("BIRDS1") 
labels = sorted(labels)
labels[(model_97.predict(tf.expand_dims(image, axis = 0))).argmax()]

1/1 [==============================] - 0s 22ms/step


"lewi's_woodpecker"

In [18]:
king_eider = utils.load_prepare_image_in_tf("images/king_eider.jpg", img_shape=224)

In [32]:
labels[(model_97.predict(tf.expand_dims(king_eider, axis = 0))).argmax() ]

1/1 [==============================] - 0s 26ms/step


'king_eider'

In [2]:
import utils
image = utils.load_prepare_image_in_tf("images/american_flamingo.jpg", img_shape=224)
image

<tf.Tensor: shape=(224, 224, 3), dtype=float32, numpy=
array([[[0.7894521 , 0.82866776, 0.8639619 ],
        [0.79651314, 0.8396504 , 0.8631798 ],
        [0.7886884 , 0.83182573, 0.85535514],
        ...,
        [0.7775209 , 0.84418756, 0.87434334],
        [0.7847482 , 0.8357286 , 0.8710227 ],
        [0.7761992 , 0.8311012 , 0.8663953 ]],

       [[0.7924137 , 0.8316294 , 0.8669235 ],
        [0.7993261 , 0.8424634 , 0.8659928 ],
        [0.7965485 , 0.83968574, 0.86321515],
        ...,
        [0.7793419 , 0.842087  , 0.89306736],
        [0.7764706 , 0.8509804 , 0.8784314 ],
        [0.7874066 , 0.84230864, 0.87760276]],

       [[0.78607315, 0.82136726, 0.8566614 ],
        [0.7903423 , 0.8334795 , 0.85700893],
        [0.79725873, 0.8364744 , 0.87176853],
        ...,
        [0.78255427, 0.8374563 , 0.8884367 ],
        [0.78431374, 0.8509804 , 0.8901961 ],
        [0.77874464, 0.8454113 , 0.87678385]],

       ...,

       [[0.7327342 , 0.80156296, 0.8355002 ],
        [0.76

In [3]:
result = model_97.predict(tf.expand_dims(image, axis=0)).argmax()
result


2024-02-11 19:32:37.369087: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
2024-02-11 19:32:37.541131: W external/local_xla/xla/stream_executor/gpu/asm_compiler.cc:225] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 8.9
2024-02-11 19:32:37.541158: W external/local_xla/xla/stream_executor/gpu/asm_compiler.cc:228] Used ptxas at ptxas
2024-02-11 19:32:37.541275: W external/local_xla/xla/stream_executor/gpu/redzone_allocator.cc:322] UNIMPLEMENTED: ptxas ptxas too old. Falling back to the driver to compile.
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2024-02-11 19:32:37.796557: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.


1/1 [==============================] - 3s 3s/step


2024-02-11 19:32:39.085718: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.


1

In [7]:
sorted(os.listdir("BIRDS1"))

['Fulvous whistling duck',
 'acorn_woodpecker',
 'american_flamingo',
 'american_three_toed_woodpecker',
 'american_wigeon',
 'amethyst_throated_mountain_gem',
 "anna's_hummingbird",
 'baikal_teal',
 'barrows_goldeneye',
 'black_bellied_whistling_duck',
 'black_chinned_hummingbird',
 'black_vulture',
 'boreal_owl',
 'brant_goose',
 'broad_billed_hummingbird',
 'broad_tailed_hummingbird',
 'buffle_head',
 'california_condor',
 'california_quail',
 'calliope_hummingbird',
 'canvas_back',
 'chukar_partridge',
 'cinnamon_teal',
 'common_goldeneye',
 'costas_hummingbird',
 'downy_woodpecker',
 'emperor_goose',
 'gray_partridge',
 'greater_prairie_chicken',
 'greater_sage_grouse',
 'greater_white_fronted_goose',
 'greylag_goose',
 'harlequin_duck',
 'hooded_merganser',
 'king_eider',
 "lewi's_woodpecker",
 'limpkin',
 'mallard_duck',
 'mexican_violetear',
 'northern_flicker',
 'northern_shoveler',
 'pileated_woodpecker',
 'pink_footed_goose',
 'red_bellied_woodpecker',
 'red_breasted_mergans

In [9]:
import os
labels = os.listdir("BIRDS1") 
labels = sorted(labels)
result_label = labels[(model_97.predict(tf.expand_dims(image, axis = 0))).argmax() ]
result_label

1/1 [==============================] - 0s 18ms/step


'american_flamingo'